In [ ]:
!pip install pandas sqlalchemy geopy sklearn geopy
!pip install urllib3

In [ ]:
import heapq
from collections import defaultdict
from geopy.distance import geodesic
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sqlalchemy import create_engine, Column, String, Integer, Float
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
import urllib.request                   # this would output the html source text
import requests                         # this request the url

In [ ]:
# Set up the database
Base = declarative_base()
engine = create_engine('sqlite:///organ_donation.db')
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:

# Donor and Recipient Classes
class Donor(Base):
    __tablename__ = 'donors'
    id = Column(String, primary_key=True)
    blood_type = Column(String)
    hla_type = Column(String)
    location_lat = Column(Float)
    location_lon = Column(Float)

class Recipient(Base):
    __tablename__ = 'recipients'
    id = Column(String, primary_key=True)
    blood_type = Column(String)
    urgency = Column(Integer)
    location_lat = Column(Float)
    location_lon = Column(Float)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
# Priority Queue to manage recipient urgency
class UrgencyQueue:
    def __init__(self):
        self.heap = []

    def add_recipient(self, recipient):
        heapq.heappush(self.heap, recipient)

    def get_highest_priority(self):
        return heapq.heappop(self.heap) if self.heap else None

In [ ]:
# Function to calculate the best donor match based on blood type and location
def find_best_match(recipient, donors):
    best_match = None
    best_distance = float('inf')

    for donor in donors:
        distance = geodesic((recipient.location_lat, recipient.location_lon), (donor.location_lat, donor.location_lon)).miles
        if distance < best_distance:
            best_distance = distance
            best_match = donor

    return best_match

In [ ]:
#Fetching the data
#This is just reading the data from URL
with urllib.request.urlopen("https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt") as url:
    data = url.read()

#This is just Writing the data from URL to text file in the file system
urllist = ["https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt" ]
for url in urllist:
    data = requests.get(url)
with open('shakespear.txt', 'w') as file:
        file.write(data.text)

In [22]:
# Load the provided Excel file to inspect its contents
file_path = '/content/Organ_Donation_and_Transplantation_Data.xlsx'
excel_data = pd.ExcelFile(file_path)

# Check the sheet names and inspect the first few rows of each sheet
sheet_names = excel_data.sheet_names
# sheet_data = {sheet: excel_data.parse(sheet).head() for sheet in sheet_names}

# sheet_names, sheet_data

sheet_names


['READ ME FIRST',
 'Overview - National',
 'Overview - State',
 'Donor Demographics',
 'OPO - POTR',
 'OPO Service Area by County',
 'Waitlist-Candidate Registration',
 'Waitlist - OTC Transplant Rate',
 'Waitlist - OTC Removal Reasons',
 'Recipient Demographics',
 'OTC Transplants by Demographics',
 'OTC Outcome Measures']

In [ ]:
data = pd.read_excel('/content/Organ_Donation_and_Transplantation_Data.xlsx')
# sheet_data = {sheet: excel_data.parse(sheet).head() for sheet in sheet_names}

In [ ]:
national = pd.read_excel(file_path, sheet_name='Overview - National')
state = pd.read_excel(file_path, sheet_name='Overview - State')
donor = pd.read_excel(file_path, sheet_name='Donor Demographics')
POTR = pd.read_excel(file_path, sheet_name='OPO - POTR')
wait_candidate = pd.read_excel(file_path, sheet_name='Waitlist-Candidate Registration')
wait_transplant_rate = pd.read_excel(file_path, sheet_name='Waitlist - OTC Transplant Rate')
wait_transplant_remove = pd.read_excel(file_path, sheet_name='Waitlist - OTC Removal Reasons')
recipent = pd.read_excel(file_path, sheet_name='Recipient Demographics')
transplant_demo = pd.read_excel(file_path, sheet_name='OTC Transplants by Demographics')
outcome = pd.read_excel(file_path, sheet_name='OTC Outcome Measures')


In [ ]:
print(national.head())

   Year            Organ Number of deceased organ donors recovered  \
0  2019            Heart                                      3636   
1  2019           Kidney                                     11152   
2  2019  Kidney-Pancreas                                         .   
3  2019            Liver                                      9153   
4  2019             Lung                                      2755   

  Number of living organ donors recovered  Number of candidates added  \
0                                       .                        4658   
1                                    6866                       41100   
2                                       .                        1421   
3                                     523                       12941   
4                                       .                        3218   

   Number of registrations added  \
0                           4717   
1                          43223   
2                           1479

In [ ]:
print(state.head())

              statistics  Year State of Residence Name  \
0  Deceased Organ Donors  2019                 Alabama   
1  Deceased Organ Donors  2019                 Alabama   
2  Deceased Organ Donors  2019                 Alabama   
3  Deceased Organ Donors  2019                 Alabama   
4  Deceased Organ Donors  2019                 Alabama   

  State of Residence Code   Organ    counts Last Refresh Date  \
0                      AL     All       206     July 16, 2024   
1                      AL   Heart        83     July 16, 2024   
2                      AL  Kidney       194     July 16, 2024   
3                      AL   Liver       177     July 16, 2024   
4                      AL    Lung        38     July 16, 2024   

  Date Range for Data - Start Date Range for Data - End  
0             January 1, 2019            March 31, 2024  
1             January 1, 2019            March 31, 2024  
2             January 1, 2019            March 31, 2024  
3             January 1, 201

In [ ]:
print(donor.head())

         Category Donor Type Filter           Level Donor Count  \
0       0 - Total       Grand Total           Total      108645   
1  1 - Donor Type    Deceased Donor  Deceased Donor       73829   
2  1 - Donor Type      Living Donor    Living Donor       34816   
3   2 - Insurance      Living Donor         Insured       32097   
4   2 - Insurance      Living Donor       Uninsured        2177   

  Last Refresh Date Date Range for Data - Start Date Range for Data - End  
0     July 16, 2024             January 1, 2019            March 31, 2024  
1     July 16, 2024             January 1, 2019            March 31, 2024  
2     July 16, 2024             January 1, 2019            March 31, 2024  
3     July 16, 2024             January 1, 2019            March 31, 2024  
4     July 16, 2024             January 1, 2019            March 31, 2024  


In [ ]:
print(POTR.head())

      OPO Name (State)  Year     Organ Procured Organ Transplant Rate  \
0  Average of All OPOs  2019     Heart                          99.3%   
1  Average of All OPOs  2019    Kidney                          80.4%   
2  Average of All OPOs  2019     Liver                          92.6%   
3  Average of All OPOs  2019      Lung                          94.6%   
4  Average of All OPOs  2019  Pancreas                          76.4%   

  Last Refresh Date Date Range for Data - Start Date Range for Data - End  
0     July 16, 2024             January 1, 2019            March 31, 2024  
1     July 16, 2024             January 1, 2019            March 31, 2024  
2     July 16, 2024             January 1, 2019            March 31, 2024  
3     July 16, 2024             January 1, 2019            March 31, 2024  
4     July 16, 2024             January 1, 2019            March 31, 2024  


In [ ]:
print(wait_candidate.head())

  Candidate/Registration  Year of Addition       Organ Type  Total  \
0             Candidates              2019              All  61164   
1             Candidates              2019            Heart   4658   
2             Candidates              2019           Kidney  41100   
3             Candidates              2019  Kidney-Pancreas   1421   
4             Candidates              2019            Liver  12941   

   Sex: Female  Sex: Male  Race: White, Non-Hispanic  \
0        23265      37908                      30913   
1         1397       3261                       2740   
2        15727      25377                      17295   
3          597        825                        704   
4         4970       7972                       8936   

   Race: Black, Non-Hispanic Race: Asian, Non-Hispanic  Race: Hispanic/Latino  \
0                      14226                      3798                  11139   
1                       1163                       155                    540   

In [ ]:
print(wait_transplant_rate.head())

           Transplant Center (State) Organ Type  \
0  Abbott Northwestern Hospital (MN)      Heart   
1  Abbott Northwestern Hospital (MN)     Kidney   
2  Abbott Northwestern Hospital (MN)        All   
3          AdventHealth Orlando (FL)      Heart   
4          AdventHealth Orlando (FL)     Kidney   

  Percent of Deceased Donor Transplant with 30 Days  \
0                                             25.6%   
1                                              2.6%   
2                                              9.0%   
3                                             44.8%   
4                                              8.4%   

  Percent of Deceased Donor Transplant with 1 Year  \
0                                            71.1%   
1                                             9.1%   
2                                            26.5%   
3                                            84.6%   
4                                            28.8%   

  Percent of Deceased Donor Transplan

In [ ]:
print(wait_transplant_remove.head())

           Transplant Center (State)       Organ Type  \
0  Abbott Northwestern Hospital (MN)            Heart   
1  Abbott Northwestern Hospital (MN)           Kidney   
2          AdventHealth Orlando (FL)            Heart   
3          AdventHealth Orlando (FL)           Kidney   
4          AdventHealth Orlando (FL)  Kidney-Pancreas   

  Transferred to Another Center Received Living Donor Transplant  \
0                             0                                0   
1                            19                               41   
2                             0                                0   
3                    Suppressed                              126   
4                             0                                0   

  Received Deceased Donor Transplant        Died  \
0                                 67  Suppressed   
1                                115          28   
2                                266  Suppressed   
3                                587    

In [ ]:
print(recipent.head())

   Transplant Year      Donor Type Organ Transplanted  \
0             2019  Deceased Donor              Heart   
1             2019  Deceased Donor             Kidney   
2             2019  Deceased Donor    Kidney-Pancreas   
3             2019  Deceased Donor              Liver   
4             2019  Deceased Donor               Lung   

   Total Distinct Recipients Male Distinct Recipients  \
0                       3542                     2459   
1                      16521                     9935   
2                        872                      545   
3                       8263                     5251   
4                       2711                     1632   

  Female Distinct Recipients  Distinct Recipients with Age Not Reported  \
0                       1083                                          0   
1                       6586                                          0   
2                        327                                          0   
3             

In [ ]:
print(transplant_demo.head())

               OTC_Name_State      Donor Type            Organ Transplanted  \
0  Children's of Alabama (AL)      All Donors  All Transplanted Organ Types   
1  Children's of Alabama (AL)  Deceased Donor  All Transplanted Organ Types   
2  Children's of Alabama (AL)  Deceased Donor                         Heart   
3  Children's of Alabama (AL)  Deceased Donor                        Kidney   
4  Children's of Alabama (AL)  Deceased Donor                         Liver   

  Total Transplants Transplants with Male Recipients  \
0               113                               67   
1                96                               56   
2                27                       Suppressed   
3                50                               30   
4                19                       Suppressed   

  Transplants with Female Recipients Pediatric Transplants  \
0                                 46                   112   
1                                 40                    96   
2 

In [ ]:
print(outcome.head())

                        OTC_Name_State                      Type  \
0           Children's of Alabama (AL)   Deceased Donor  - Heart   
1           Children's of Alabama (AL)  Deceased Donor  - Kidney   
2           Children's of Alabama (AL)   Deceased Donor  - Liver   
3           Children's of Alabama (AL)  Living Donor    - Kidney   
4  University of Alabama Hospital (AL)   Deceased Donor  - Heart   

  Organ Transplanted Percent of Transplants with Functioning Graft at 30 Days  \
0              Heart                                            100.00%         
1             Kidney                                            100.00%         
2              Liver                                            100.00%         
3             Kidney                                            100.00%         
4              Heart                                             97.78%         

  Percent of Transplants with Functioning Graft at 1 year  \
0                                          